In [1]:
# -*- coding: utf-8 -*-
"""
Created on May 31 2021
@author: Te-Yu,Lu
"""

from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
import gc
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
from collections import Counter
from tqdm import tqdm


pd.set_option("display.max_rows", 1000)    #設定最大能顯示1000rows
pd.set_option("display.max_columns", 1000)

# import keras

# 載入 ckiptagger 自訂詞典
User_Dict = {}
with open("KccLy2Dict2021.txt","r", encoding='utf-8-sig') as UDicts:
    for udic in UDicts:
        udWord = udic.strip().split(" ")
        if len(udWord) > 1:
            User_Dict[udWord[0]] = udWord[1]
        else:
            User_Dict[udWord[0]] = 100    # 未給定權重者一律賦予預設值 150                
dictionary = construct_dictionary(User_Dict)

# 載入模型
ws = WS("../data")
# pos = POS("./data")
# ner = NER("./data")

# 載入 StopWord
stopword_list = []
with open("../dict/KccStopWord2020.txt","r", encoding='utf-8-sig') as stopwords:
    for stopword in stopwords:
        stopword_list.append(stopword.strip())

FileNotFoundError: [Errno 2] No such file or directory: '../data/embedding_character/token_list.npy'

In [ ]:
word = pd.read_csv('委員逐字稿.csv',encoding='utf-8') #匯資料
lin = word[word.Name == "林淑芬"]
lin.head(1)

In [ ]:
title_lin = lin[['Speech']] #提出會用到的欄位
title_lin.head(2)

In [ ]:
lin_list= list(title_lin['Speech'])

lin_str =(','.join('%s' %id for id in lin_list)) 
#print(lin_str)

In [ ]:
# 開始斷詞
# sentence_segmentation = True,  => To consider delimiters
# segment_delimiter_set = {",", "。", ":", "?", "!", ";"}), # This is the defualt set of delimiters
# recommend_dictionary = dictionary1, # words in this dictionary are encouraged
# coerce_dictionary = dictionary2, # words in this dictionary are forced
strCut = ws([lin_str], sentence_segmentation=True,
                 segment_delimiter_set = {'"',"\r\n","，","「","」",",","？",'\n', "。", ":", "?", "!", ";", "、"},
                 coerce_dictionary = dictionary)


print("------------CKIPTagger Word Segment ws(str,....) + 刪除標點及停用字 ---------------")
stayed_lst= []
for word in strCut[0]:
     # 刪除停用字
    if word not in stopword_list:
        stayed_lst.append(word)

print(" / ".join(stayed_lst))

In [ ]:
# -*- coding: utf-8 -*-
"""
Input: "KCC Data/CkipNewsCisTest-KccDict2020.txt"
Output: "KCC Data/NewsCisTest-KccDict2020-LDA-TopicNum10.txt"
#CkipNewsCisTest-KccDict2020.txt

@author: Johnson
"""
# When numpy version = 1.19.2 => gensim version should downgrade to 3.8.3
from gensim import corpora, models, utils
from gensim.models import LdaModel, CoherenceModel

In [ ]:
import numpy
stayed_lst_new = [d.split() for d in stayed_lst]
dict = corpora.Dictionary(numpy.asarray(stayed_lst_new))
print("dict = ", dict)
print("------------------------------")

In [ ]:
# creat mapping matrix of [word_id, word_frequency] for each word segment
corpus = [dict.doc2bow(text) for text in stayed_lst_new]   
print("corpus[:1] = ", corpus[:1])   
print("------------------------------")

print("print lists of [word:frequency]")
[[print((dict[id], freq)) for id,freq in cp] for cp in corpus[:5]]
print("======================================================")

In [ ]:
lda_model = models.ldamodel.LdaModel(corpus=corpus, id2word= dict, num_topics=10, passes=20, alpha='auto', chunksize=50, per_word_topics=True)
# print(ldamodel.print_topics())   列出最重要的前若干个主题

output = open("2_NewsCisTest-KccDict2020-LDA-TopicNum10.txt", 'w', encoding='utf-8-sig')
lstTM = lda_model.print_topics(num_topics=10, num_words=20)
for tupleTM in lstTM:
        str1 = "".join(str(x) for x in tupleTM)
        #print("type(str1) = ", type(str1))
        str1 += "\n"
        print("strTM = ", str1)
        output.write(str1)
output.close()
 
print("------------------------------")
print("Perplexity = ", lda_model.log_perplexity(corpus))

print("------------------------------")
cm = CoherenceModel(model=lda_model, texts=list(stayed_lst_new), dictionary=dict, coherence= 'c_v' )
coher_lda = cm.get_coherence()
print("\n Coherence Score: ", coher_lda)